# Surrounding Venues for Each Arcade

__Skip to the Good Stuff__

[Importing Libraries](#Importing-Libraries)

[Loading Arcade DataFrame](#Loading-Arcade-DataFrame)

[Connecting to FourSquare](#Connecting-to-FourSquare)

[Making Surrounding Venues DataFrame](#Making-Surrounding-Venues-DataFrame)

[Dropping the Nulls](#Dropping-the-Nulls)

[Sorting by Arcade and Distance](#Sorting-by-Arcade-and-Distance)

[Ensuring Unique Venues](#Ensuring-Unique-Venues)

[Pulling the Ratings](#Pulling-the-Ratings)

### Importing Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

! pip install folium==0.5.0
import folium # plotting library

import json

pd.set_option('display.max_rows', None) #To view the full dataframe

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Loading Arcade DataFrame

In [2]:
atlanta_arcades = pd.read_csv('Arcade_final_clean.csv')
atlanta_arcades = atlanta_arcades.drop('Unnamed: 0', axis = 1)
print(len(atlanta_arcades))
atlanta_arcades

7


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Wildwood Forest,4c02e10d187ec92870e5b47b,Duffer's,1883,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf,8.5
1,Old Fourth Ward,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,792,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office,Not Rated
2,Summerhill,4d7412d3d6f06dcbbdb70f08,Foege's Arcade,428,No Address,33.740652,-84.384216,4bf58dd8d48988d1e1931735,Arcade,Not Rated
3,Lake Clair,4f5c6a22e4b0a3bd23833b66,Marlin Arcade,1142,No Address,38.495137,-90.007739,4bf58dd8d48988d11b941735,Pub,Not Rated
4,Downtown,503e646b16488fc60417dc31,PAC-MAN PLAY™ Arcade at Underground Atlanta,322,50 Upper Alabama Street,33.752497,-84.390085,4bf58dd8d48988d1e1931735,Arcade,Not Rated
5,Midtown,552d475d498e4d32bce2696f,Atlanta Arcade & Gaming Rentals,649,925B Peachtree St NE,33.785318,-84.378605,4bf58dd8d48988d1e1931735,Arcade,Not Rated
6,Virginia Highland,5b04dca96bd36b002c157123,Atlanta Arcade Rentals,1378,931 Monroe Dr NE #201,33.779940,-84.368230,5454152e498ef71e2b9132c6,Event Service,Not Rated


### Connecting to FourSquare

In [3]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [4]:
VERSION = '20180604'
LIMIT = 20

### Making Surrounding Venues DataFrame

In [5]:
#assign variables for the search
search_query = ''
radius = 50
print(search_query + ' .... OK!')

 .... OK!


In [6]:
surrounding_venues = pd.DataFrame(columns = ['Arcade','Venue_ID','Venue_Name','Venue_Distance','Venue_Address','Venue_Latitude','Venue_Longitude','Category_ID','Category_Name'])
surrounding_venues

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name


In [7]:
for index, arcade in enumerate(atlanta_arcades['Venue_ID']):
    try:
        arcade_name = atlanta_arcades['Venue_Name'][index]
        latitude = atlanta_arcades['Venue_Latitude'][index]
        longitude = atlanta_arcades['Venue_Longitude'][index]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
    except:
        continue
    try:
        res = results['response']['venues']
    except:
        continue
    for dictionary in results['response']['venues']:
        try:
            ident= dictionary['id']
        except:
            ident= 'No ID'
        try:
            name = dictionary['name']
        except:
            name = 'No Name'
        try:
            ven_add = dictionary['location']['address']
        except:
            ven_add = 'No Address'
        try:
            ven_lat = dictionary['location']['lat']
        except:
            ven_lat = 'No Latitude'
        try:
            ven_lng = dictionary['location']['lng']
        except:
            ven_lng = 'No Longitude'
        try:
            ven_dist = dictionary['location']['distance']
        except:
            ven_dist = 'No Distance'
        try:
            for cat in dictionary['categories']:
                try:
                    cat_id= cat['id']
                except:
                    cat_id= 'No ID'
                try:
                    cat_name = cat['name']
                except:
                    cat_name = 'No Name'
        except:
            continue
        try:
            surrounding_venues = surrounding_venues.append({'Venue_Distance':ven_dist,'Arcade':arcade_name, 'Venue_ID':ident, 'Venue_Name':name,'Venue_Address':ven_add,'Venue_Latitude':ven_lat,'Venue_Longitude':ven_lng,'Category_ID':cat_id,'Category_Name':cat_name}, ignore_index=True)
        except:
            surrounding_venues = surrounding_venues.append({'Arcade': 'Null'}, ignore_index=True)

In [8]:
print(len(surrounding_venues))
surrounding_venues.head()

140


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Duffer's,4c02e10d187ec92870e5b47b,Duffer's,0,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf
1,Duffer's,60ee10fd77663d332252d1b1,Tara’s,33,No Address,38.980954,-74.825312,4bf58dd8d48988d1c3941735,Moroccan Restaurant
2,Duffer's,4bf9c2f45ec320a105d28ad3,Dominics Place,31,5209-5299 Pacific Ave,38.980841,-74.825380,4bf58dd8d48988d1c3941735,Moroccan Restaurant
3,Duffer's,4e73c0ea52b145ffbfb7a5ac,Eden roc,262,Bennett,38.979571,-74.823254,4bf58dd8d48988d1fb931735,Motel
4,Duffer's,503abac4e4b0fad35e545b9b,"One Fish, Two Fish Restaurant",40,5209 Pacific Ave,38.980910,-74.825234,4bf58dd8d48988d157941735,New American Restaurant


In [9]:
surrounding_venues.to_csv('surrounding_raw.csv')

### Dropping the Nulls

In [10]:
surrounding_venues = pd.read_csv('surrounding_raw.csv')
surrounding_venues = surrounding_venues.drop('Unnamed: 0', axis= 1)
print(len(surrounding_venues))
surrounding_venues.head()

140


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Duffer's,4c02e10d187ec92870e5b47b,Duffer's,0,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf
1,Duffer's,60ee10fd77663d332252d1b1,Tara’s,33,No Address,38.980954,-74.825312,4bf58dd8d48988d1c3941735,Moroccan Restaurant
2,Duffer's,4bf9c2f45ec320a105d28ad3,Dominics Place,31,5209-5299 Pacific Ave,38.980841,-74.825380,4bf58dd8d48988d1c3941735,Moroccan Restaurant
3,Duffer's,4e73c0ea52b145ffbfb7a5ac,Eden roc,262,Bennett,38.979571,-74.823254,4bf58dd8d48988d1fb931735,Motel
4,Duffer's,503abac4e4b0fad35e545b9b,"One Fish, Two Fish Restaurant",40,5209 Pacific Ave,38.980910,-74.825234,4bf58dd8d48988d157941735,New American Restaurant


In [11]:
surrounding_venues = surrounding_venues[surrounding_venues != 'Null']
surrounding_venues = surrounding_venues[surrounding_venues != '[]']
surrounding_venues = surrounding_venues.dropna()
print(len(surrounding_venues))
surrounding_venues = surrounding_venues.reset_index()
surrounding_venues = surrounding_venues.drop(['index'], axis = 1)
surrounding_venues.head(20)

140


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Duffer's,4c02e10d187ec92870e5b47b,Duffer's,0,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf
1,Duffer's,60ee10fd77663d332252d1b1,Tara’s,33,No Address,38.980954,-74.825312,4bf58dd8d48988d1c3941735,Moroccan Restaurant
2,Duffer's,4bf9c2f45ec320a105d28ad3,Dominics Place,31,5209-5299 Pacific Ave,38.980841,-74.825380,4bf58dd8d48988d1c3941735,Moroccan Restaurant
3,Duffer's,4e73c0ea52b145ffbfb7a5ac,Eden roc,262,Bennett,38.979571,-74.823254,4bf58dd8d48988d1fb931735,Motel
4,Duffer's,503abac4e4b0fad35e545b9b,"One Fish, Two Fish Restaurant",40,5209 Pacific Ave,38.980910,-74.825234,4bf58dd8d48988d157941735,New American Restaurant
5,Duffer's,4c660b8ef7a80f47324c9d7a,Green's Liquors,48,5301 Pacific Ave,38.980536,-74.825765,4bf58dd8d48988d186941735,Liquor Store
6,Duffer's,51cf8c2a498e603aa0e99439,Duffers Mini Golf,28,No Address,38.981139,-74.825453,4bf58dd8d48988d1e6941735,Golf Course
7,Duffer's,4fc0f30ce4b07e3cd94ac219,Clam Casino,136,No Address,38.980406,-74.824298,4bf58dd8d48988d17c941735,Casino
8,Duffer's,4fd5bf40e4b0b56919bc3526,Forget-Me-Not Beach,123,No Address,38.980877,-74.827121,4bf58dd8d48988d1e3941735,Surf Spot
9,Duffer's,50018596e4b0eed3b6f65639,The Beach at Hildreth,59,No Address,38.980644,-74.825162,4bf58dd8d48988d1e3941735,Surf Spot


In [12]:
count = -1
distance_drop_list = list()
for dist in surrounding_venues['Venue_Distance']:
    count = count + 1
    if dist == 0:
        distance_drop_list.append(count)
print(len(distance_drop_list))

6


In [13]:
surrounding_venues = surrounding_venues.drop(index = distance_drop_list)
print(len(surrounding_venues))
surrounding_venues.head()

134


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
1,Duffer's,60ee10fd77663d332252d1b1,Tara’s,33,No Address,38.980954,-74.825312,4bf58dd8d48988d1c3941735,Moroccan Restaurant
2,Duffer's,4bf9c2f45ec320a105d28ad3,Dominics Place,31,5209-5299 Pacific Ave,38.980841,-74.825380,4bf58dd8d48988d1c3941735,Moroccan Restaurant
3,Duffer's,4e73c0ea52b145ffbfb7a5ac,Eden roc,262,Bennett,38.979571,-74.823254,4bf58dd8d48988d1fb931735,Motel
4,Duffer's,503abac4e4b0fad35e545b9b,"One Fish, Two Fish Restaurant",40,5209 Pacific Ave,38.980910,-74.825234,4bf58dd8d48988d157941735,New American Restaurant
5,Duffer's,4c660b8ef7a80f47324c9d7a,Green's Liquors,48,5301 Pacific Ave,38.980536,-74.825765,4bf58dd8d48988d186941735,Liquor Store


### Sorting by Arcade and Venue Distance

In [14]:
surrounding_venues = surrounding_venues.sort_values(['Arcade', 'Venue_Distance'],ascending = True)
surrounding_venues.head(10)

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
27,Arcade 160 Studios,4dc014671e72dd48b20019a6,MW Law Atlanta,6,621 North Ave,33.770113,-84.367077,4bf58dd8d48988d174941735,Coworking Space
28,Arcade 160 Studios,57bde049498e3cc1f7d51ae2,North Creative,20,"621 North Ave NE, Suite C170",33.770279,-84.367119,4bf58dd8d48988d1f1931735,General Entertainment
31,Arcade 160 Studios,52614d2e11d2226cbe1a8e3b,Levine City Real Estate,20,621 North Ave NE Ste C50,33.770279,-84.367119,5032885091d4c4b30a586d66,Real Estate Office
22,Arcade 160 Studios,4a7cb042f964a52041ed1fe3,North Ave Post,23,621 North Ave,33.770274,-84.367291,4d954b06a243a5684965b473,Residential Building (Apartment / Condo)
29,Arcade 160 Studios,4b548145f964a520bebd27e3,Keller Williams,23,621 North Ave NE Ste C50,33.770262,-84.366989,5032885091d4c4b30a586d66,Real Estate Office
21,Arcade 160 Studios,4d38948f6eef5481d4ee4148,"McManamy, McLeod, Heller",32,621 North Ave NE,33.770229,-84.366834,4bf58dd8d48988d127941735,Conference Room
20,Arcade 160 Studios,4c00162dc30a2d7f54fc111d,Southern Dairies,34,No Address,33.770381,-84.366980,4bf58dd8d48988d130941735,Building
24,Arcade 160 Studios,4c40b53da5c5ef3b2710b06f,Cardlytics,36,621 North Ave,33.770395,-84.366974,4bf58dd8d48988d124941735,Office
32,Arcade 160 Studios,5d3f7c6a91896e0008d83e70,The Lola,39,621 North Ave NE Ste D100,33.769911,-84.367508,4bf58dd8d48988d174941735,Coworking Space
25,Arcade 160 Studios,5a0124f393bd634fafe63468,Deep End,61,621 North Ave NE Ste A100,33.770622,-84.366943,56aa371ae4b08b9a8d5734ba,Tex-Mex Restaurant


In [15]:
surrounding_venues.to_csv('surrounding_venues_sorted.csv')

### Ensuring Unique Venues

In [16]:
surrounding_sorted = pd.read_csv('surrounding_venues_sorted.csv')
surrounding_sorted = surrounding_sorted.drop('Unnamed: 0', axis= 1)
print(len(surrounding_sorted))
surrounding_sorted.head(20)

134


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Arcade 160 Studios,4dc014671e72dd48b20019a6,MW Law Atlanta,6,621 North Ave,33.770113,-84.367077,4bf58dd8d48988d174941735,Coworking Space
1,Arcade 160 Studios,57bde049498e3cc1f7d51ae2,North Creative,20,"621 North Ave NE, Suite C170",33.770279,-84.367119,4bf58dd8d48988d1f1931735,General Entertainment
2,Arcade 160 Studios,52614d2e11d2226cbe1a8e3b,Levine City Real Estate,20,621 North Ave NE Ste C50,33.770279,-84.367119,5032885091d4c4b30a586d66,Real Estate Office
3,Arcade 160 Studios,4a7cb042f964a52041ed1fe3,North Ave Post,23,621 North Ave,33.770274,-84.367291,4d954b06a243a5684965b473,Residential Building (Apartment / Condo)
4,Arcade 160 Studios,4b548145f964a520bebd27e3,Keller Williams,23,621 North Ave NE Ste C50,33.770262,-84.366989,5032885091d4c4b30a586d66,Real Estate Office
5,Arcade 160 Studios,4d38948f6eef5481d4ee4148,"McManamy, McLeod, Heller",32,621 North Ave NE,33.770229,-84.366834,4bf58dd8d48988d127941735,Conference Room
6,Arcade 160 Studios,4c00162dc30a2d7f54fc111d,Southern Dairies,34,No Address,33.770381,-84.366980,4bf58dd8d48988d130941735,Building
7,Arcade 160 Studios,4c40b53da5c5ef3b2710b06f,Cardlytics,36,621 North Ave,33.770395,-84.366974,4bf58dd8d48988d124941735,Office
8,Arcade 160 Studios,5d3f7c6a91896e0008d83e70,The Lola,39,621 North Ave NE Ste D100,33.769911,-84.367508,4bf58dd8d48988d174941735,Coworking Space
9,Arcade 160 Studios,5a0124f393bd634fafe63468,Deep End,61,621 North Ave NE Ste A100,33.770622,-84.366943,56aa371ae4b08b9a8d5734ba,Tex-Mex Restaurant


### Pulling the Ratings

In [17]:
count = -1
rating_list = list()
for ident in surrounding_sorted['Venue_ID']:
    count = count +1
    venue_id = surrounding_sorted['Venue_ID'][count]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'Not Rated'
    rating_list.append(rating)
rating_list

['Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 5.4,
 'Not Rated',
 'Not Rated',
 7.3,
 8.7,
 'Not Rated',
 9.0,
 8.0,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 8.2,
 'Not Rated',
 8.4,
 'Not Rated',
 'Not Rated',
 7.7,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 9.5,
 7.4,
 7.7,
 'Not Rated',
 7.1,
 8.3,
 7.6,
 9.1,
 'Not Rated',
 8.6,
 'Not Rated',
 'Not Rated',
 7.1,
 7.5,
 'Not Rated',
 'Not Rated',
 8.5,
 8.3,
 'Not Rated',
 9.5,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 8.1,
 6.4,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 8.3,
 8.2,
 'Not Rated',
 'Not Rate

In [18]:
surrounding_final = surrounding_sorted.assign(Venue_Rating= rating_list)
print(len(surrounding_final))
surrounding_final.head()

134


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Arcade 160 Studios,4dc014671e72dd48b20019a6,MW Law Atlanta,6,621 North Ave,33.770113,-84.367077,4bf58dd8d48988d174941735,Coworking Space,Not Rated
1,Arcade 160 Studios,57bde049498e3cc1f7d51ae2,North Creative,20,"621 North Ave NE, Suite C170",33.770279,-84.367119,4bf58dd8d48988d1f1931735,General Entertainment,Not Rated
2,Arcade 160 Studios,52614d2e11d2226cbe1a8e3b,Levine City Real Estate,20,621 North Ave NE Ste C50,33.770279,-84.367119,5032885091d4c4b30a586d66,Real Estate Office,Not Rated
3,Arcade 160 Studios,4a7cb042f964a52041ed1fe3,North Ave Post,23,621 North Ave,33.770274,-84.367291,4d954b06a243a5684965b473,Residential Building (Apartment / Condo),Not Rated
4,Arcade 160 Studios,4b548145f964a520bebd27e3,Keller Williams,23,621 North Ave NE Ste C50,33.770262,-84.366989,5032885091d4c4b30a586d66,Real Estate Office,Not Rated


In [19]:
counts = dict()
for category in surrounding_final['Category_Name']:
    counts[category]= counts.get(category, 0) + 1

In [20]:
counts

{'Coworking Space': 4,
 'General Entertainment': 2,
 'Real Estate Office': 2,
 'Residential Building (Apartment / Condo)': 15,
 'Conference Room': 1,
 'Building': 2,
 'Office': 2,
 'Tex-Mex Restaurant': 1,
 'Bookstore': 1,
 'Salon / Barbershop': 3,
 'Wine Bar': 1,
 'Coffee Shop': 3,
 'Salad Place': 1,
 'Mountain': 1,
 'American Restaurant': 1,
 'Amphitheater': 1,
 'Playground': 1,
 'Food Truck': 3,
 'Board Shop': 1,
 'Monument / Landmark': 3,
 'Park': 7,
 'Gym / Fitness Center': 1,
 'Other Great Outdoors': 1,
 'Performing Arts Venue': 1,
 'Gift Shop': 2,
 'Taiwanese Restaurant': 1,
 'Poke Place': 2,
 'Lounge': 1,
 'Soup Place': 1,
 'Shopping Mall': 2,
 'Grocery Store': 1,
 'Ice Cream Shop': 2,
 'Pizza Place': 2,
 'Hot Dog Joint': 2,
 'Food & Drink Shop': 1,
 "Dentist's Office": 1,
 'Sandwich Place': 1,
 'Trail': 3,
 'Golf Course': 1,
 'Moroccan Restaurant': 2,
 'New American Restaurant': 1,
 'Liquor Store': 1,
 'Surf Spot': 2,
 'Casino': 1,
 'Pool': 3,
 'Trade School': 1,
 'Resort': 1,

In [21]:
surrounding_final.to_csv('surrounding_final.csv')

### Prepping for Clustering

In [2]:
surrounding_split = pd.read_csv('surrounding_final.csv')
surrounding_split = surrounding_split.drop('Unnamed: 0', axis = 1)
print(len(surrounding_split))
surrounding_split.head()

134


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Arcade 160 Studios,4dc014671e72dd48b20019a6,MW Law Atlanta,6,621 North Ave,33.770113,-84.367077,4bf58dd8d48988d174941735,Coworking Space,Not Rated
1,Arcade 160 Studios,57bde049498e3cc1f7d51ae2,North Creative,20,"621 North Ave NE, Suite C170",33.770279,-84.367119,4bf58dd8d48988d1f1931735,General Entertainment,Not Rated
2,Arcade 160 Studios,52614d2e11d2226cbe1a8e3b,Levine City Real Estate,20,621 North Ave NE Ste C50,33.770279,-84.367119,5032885091d4c4b30a586d66,Real Estate Office,Not Rated
3,Arcade 160 Studios,4a7cb042f964a52041ed1fe3,North Ave Post,23,621 North Ave,33.770274,-84.367291,4d954b06a243a5684965b473,Residential Building (Apartment / Condo),Not Rated
4,Arcade 160 Studios,4b548145f964a520bebd27e3,Keller Williams,23,621 North Ave NE Ste C50,33.770262,-84.366989,5032885091d4c4b30a586d66,Real Estate Office,Not Rated


In [3]:
venue_clustering_df = pd.DataFrame(surrounding_split[['Venue_Distance', 'Category_ID', 'Category_Name', 'Venue_Rating']])
venue_clustering_df.head()

,Venue_Distance,Category_ID,Category_Name,Venue_Rating
0,6,4bf58dd8d48988d174941735,Coworking Space,Not Rated
1,20,4bf58dd8d48988d1f1931735,General Entertainment,Not Rated
2,20,5032885091d4c4b30a586d66,Real Estate Office,Not Rated
3,23,4d954b06a243a5684965b473,Residential Building (Apartment / Condo),Not Rated
4,23,5032885091d4c4b30a586d66,Real Estate Office,Not Rated


In [4]:
venue_clustering_df.to_csv('atlanta_clustering_category.csv')

In [5]:
arcade_venue_df = pd.DataFrame(surrounding_split[['Arcade', 'Venue_ID', 'Venue_Name', 'Venue_Address']])
arcade_venue_df.insert(0, 'City', 'Null')
arcade_venue_df = arcade_venue_df.assign(City = 'Atlanta, GA')
arcade_venue_df.head()

,City,Arcade,Venue_ID,Venue_Name,Venue_Address
0,"Atlanta, GA",Arcade 160 Studios,4dc014671e72dd48b20019a6,MW Law Atlanta,621 North Ave
1,"Atlanta, GA",Arcade 160 Studios,57bde049498e3cc1f7d51ae2,North Creative,"621 North Ave NE, Suite C170"
2,"Atlanta, GA",Arcade 160 Studios,52614d2e11d2226cbe1a8e3b,Levine City Real Estate,621 North Ave NE Ste C50
3,"Atlanta, GA",Arcade 160 Studios,4a7cb042f964a52041ed1fe3,North Ave Post,621 North Ave
4,"Atlanta, GA",Arcade 160 Studios,4b548145f964a520bebd27e3,Keller Williams,621 North Ave NE Ste C50


In [6]:
arcade_venue_df.to_csv('atlanta_no_cluster_arcade_data.csv')